# pipeline

Angeregt durch ein Video [`Untitled12.ipynb` von Vincent D. Warmerdam](https://www.youtube.com/watch?v=yXGCKqo5cEY) über den besseren Aufbau von Notebooks mit pandas

## Vergleich Ansätze für RKI Corona Daten

### Standardansatz

In [1]:
import pandas as pd

# Daten einlesen
link="https://opendata.arcgis.com/datasets/dd4580c810204019a7b8eb3e0b329dd6_0.csv"
RKI_komplett = pd.read_csv(link)

# Fälle für Gesamtdeutschland bestimmen
RKI_Deutschland = RKI_komplett.groupby(['Meldedatum']).agg({'AnzahlFall': ['sum']}).reset_index()
RKI_Deutschland['Meldedatum'] = pd.to_datetime(RKI_Deutschland['Meldedatum'],format='%Y/%m/%d')
RKI_Deutschland.columns = ['Meldedatum','Neue Fallzahlen']
RKI_Deutschland["Neue Fallzahlen"] = RKI_Deutschland["Neue Fallzahlen"].astype(int)
RKI_Deutschland["Neue Fallzahlen Mittelwert (7 Tage)"] = RKI_Deutschland["Neue Fallzahlen"].rolling(7).mean().fillna(0)
RKI_Deutschland["Neue Fallzahlen Mittelwert (7 Tage)"] = RKI_Deutschland["Neue Fallzahlen Mittelwert (7 Tage)"].astype(int)
RKI_Deutschland["Faelle gesamt"] = RKI_Deutschland["Neue Fallzahlen"].cumsum(axis = 0) 
RKI_Deutschland.to_csv("RKI_Corona_Deutschland.csv", index=False)

# Bundesländer rausfiltern
RKI_Bundesland = RKI_komplett.groupby(['Meldedatum','Bundesland']).agg({'AnzahlFall': ['sum']}).reset_index()
RKI_Bundesland.columns=["Meldedatum","Bundesland","Neue Fallzahlen"]
RKI_Bundesland['Meldedatum'] = pd.to_datetime(RKI_Bundesland['Meldedatum'],format='%Y/%m/%d')
RKI_Bundesland.to_csv("RKI_Corona_Bundeslaender.csv", index=False)

,Meldedatum,Bundesland,Neue Fallzahlen
0,2020-01-07 00:00:00+00:00,Nordrhein-Westfalen,1
1,2020-01-19 00:00:00+00:00,Berlin,1
2,2020-01-23 00:00:00+00:00,Saarland,1
3,2020-01-25 00:00:00+00:00,Nordrhein-Westfalen,1
4,2020-01-28 00:00:00+00:00,Bayern,2
...,...,...,...
6682,2021-04-30 00:00:00+00:00,Saarland,192
6683,2021-04-30 00:00:00+00:00,Sachsen,545
6684,2021-04-30 00:00:00+00:00,Sachsen-Anhalt,345
6685,2021-04-30 00:00:00+00:00,Schleswig-Holstein,259


### pipeline Ansatz

In [11]:
import pandas as pd

link="https://opendata.arcgis.com/datasets/dd4580c810204019a7b8eb3e0b329dd6_0.csv"
RKI_komplett = pd.read_csv(link)

def start_pipeline(dataf):
    return dataf.copy()

def Summe_Faelle_Deutschland_pro_Tag(dataf):
    dataf = dataf.groupby(['Meldedatum']).agg({'AnzahlFall': ['sum']}).reset_index()
    dataf['Meldedatum'] = pd.to_datetime(dataf['Meldedatum'],format='%Y/%m/%d')
    dataf.columns = ['Meldedatum','Neue Fallzahlen']
    dataf["Neue Fallzahlen"] = dataf["Neue Fallzahlen"].astype(int)
    return dataf

def Summe_Faelle_Bundeslaender_pro_Tag(dataf):
    dataf = dataf.groupby(['Meldedatum','Bundesland']).agg({'AnzahlFall': ['sum']}).reset_index()
    dataf.columns=["Meldedatum","Bundesland","Neue Fallzahlen"]
    dataf['Meldedatum'] = pd.to_datetime(dataf['Meldedatum'],format='%Y/%m/%d')
    return dataf

def Faelle_kumuliert(dataf):
    dataf["Faelle gesamt"] = dataf["Neue Fallzahlen"].cumsum(axis = 0) 
    return dataf

def Faelle_rollenderMittelwert7Tage(dataf):    
    dataf["Neue Fallzahlen Mittelwert (7 Tage)"] = dataf["Neue Fallzahlen"].rolling(7).mean().fillna(0)
    dataf["Neue Fallzahlen Mittelwert (7 Tage)"] = dataf["Neue Fallzahlen Mittelwert (7 Tage)"].astype(int)
    return dataf

RKI_Deutschland=(RKI_komplett
 .pipe(start_pipeline)
 .pipe(Summe_Faelle_Deutschland_pro_Tag)
 .pipe(Faelle_kumuliert)
 .pipe(Faelle_rollenderMittelwert7Tage))

RKI_Bundesland=(RKI_komplett
 .pipe(start_pipeline)
 .pipe(Summe_Faelle_Bundeslaender_pro_Tag)

In [12]:
RKI_Bundesland

,Meldedatum,Bundesland,Neue Fallzahlen,Faelle gesamt,Neue Fallzahlen Mittelwert (7 Tage)
0,2020-01-07 00:00:00+00:00,Nordrhein-Westfalen,1,1,0
1,2020-01-19 00:00:00+00:00,Berlin,1,2,0
2,2020-01-23 00:00:00+00:00,Saarland,1,3,0
3,2020-01-25 00:00:00+00:00,Nordrhein-Westfalen,1,4,0
4,2020-01-28 00:00:00+00:00,Bayern,2,6,0
...,...,...,...,...,...
6682,2021-04-30 00:00:00+00:00,Saarland,192,3398088,928
6683,2021-04-30 00:00:00+00:00,Sachsen,545,3398633,1006
6684,2021-04-30 00:00:00+00:00,Sachsen-Anhalt,345,3398978,875
6685,2021-04-30 00:00:00+00:00,Schleswig-Holstein,259,3399237,871


## Vergleich Ansätze für Wetterdaten